### Here I collect data from kaz&rus dialogues (very small, but good for domain)

In [59]:
import requests
from bs4 import BeautifulSoup as BS

import pandas as pd
import re
import os

In [60]:
directory = os.getcwd()
raw_datadir = directory+r'/raw_data'

In [46]:
# Parsing text from website with dialogues

r = requests.get('http://testent.ru/load/ucheniku/kazakhskij_jazyk/audio_dialogi_na_kazahskom_jazyke_s_perevodom/26-1-0-3871')
html = BS(r.content, 'html.parser')

#entire text  
text = []
for i in range(len(html.span) + 1):
    text.append(str(html.select('span')[i]))

fullText = text

#splitting into kazakh and russian
kazDialog = []
rusDialog = []
for i in range(len(fullText)):
    dialog = re.split('\<br\/\>\<br\/\>\<br\/\>', fullText[i])
    if (len(dialog) == 2):
        kazDialog.append(dialog[0])
        rusDialog.append(dialog[1])
        
kazList = []
kazLang = []
for dlg in kazDialog:
    sentences = re.split('\.', dlg)
    kazList.append(sentences)
    kazSentences = [item for sublist in kazList for item in sublist]
    
ruList = []
ruLang = []
for dlg in rusDialog:
    sentences = re.split('\.', dlg)
    ruList.append(sentences)
    ruSentences = [item for sublist in ruList for item in sublist]

for i in range(len(ruSentences)):
    ruLang.append('Russian')
for i in range(len(kazSentences)):
    kazLang.append('Kazakh')    
# kazSenteces & ruSentences are sentences I have parsed from a website
kazData = pd.DataFrame({'Text':kazSentences,
                        'Language': kazLang})
ruData = pd.DataFrame({'Text': ruSentences,
                       'Language': ruLang})

In [47]:
kazData

,Text,Language
0,<span>- Сәлеметсіздер ме! Наурыз құтты болсын!...,Kazakh
1,Мен кешігіп қалдым,Kazakh
2,"Кешіріңіздер!<br/>- Сәлеметсіз бе, Мұрат! Рақ...",Kazakh
3,"Мұрат, наурыз көжеден дәм татыңыз!<br/>- Наур...",Kazakh
4,<br/>- Рақмет! Тойдым,Kazakh
...,...,...
412,<br/>- Сен мұның бәрін қайдан білесің?<br/>- Ж...,Kazakh
413,"<br/>- Ә-ә, солай де",Kazakh
414,"Олай болса, Қазақстан Республикасының Констит...",Kazakh
415,<br/>Жаңағы мен айтып өткен азаматтардың құқы...,Kazakh


In [26]:
import string

In [50]:
translate_table = dict((ord(char), None) for char in string.punctuation)
translate_table[ord('«')] = None
translate_table[ord('»')] = None
translate_table[ord('–')] = None
translate_table[ord('•')] = None
translate_table[ord('№')] = None
translate_table[ord('—')] = None
translate_table[ord('“')] = None
translate_table[ord('²')] = None
translate_table[ord('í')] = None
translate_table[ord('ö')] = None
translate_table[ord('ı')] = None
translate_table[ord('−')] = None
translate_table[ord('…')] = None

kazDataPrep = []
kazLang = []
for i, line in kazData.iterrows():
    line = line['Text']
    if len(line)!=0:
        line = line.lower()
        line = re.sub(r'\d+','',line)
        line = re.sub(r'[a-zA-Z]+','',line) #removing all english characters
        line = line.translate(translate_table) 
        line = re.sub(r'^\\u$[a-zA-Z]','', line) #removing special characters from wikipedia articles
        line = re.sub(r'\ufeff','', line) #removing special characters from wikipedia articles
        line = re.sub(r'\u200b','', line) #removing special characters from wikipedia articles
        line = re.sub(r'[\u4e00-\u9fff]+','', line) #removing special characters from wikipedia articles
        line = "".join(c for c in line if c.isalpha() or c == " ")
        kazDataPrep.append(line)
        kazLang.append('Kazakh')

ruDataPrep = []
ruLang = []
for i, line in ruData.iterrows():
    line = line['Text']
    if len(line)!=0:
        line = line.lower()
        line = re.sub(r'\d+','',line)
        line = re.sub(r'[a-zA-Z]+','',line) #removing all english characters
        line = line.translate(translate_table) 
        line = re.sub(r'^\\u$[a-zA-Z]','', line) #removing special characters from wikipedia articles
        line = re.sub(r'\ufeff','', line) #removing special characters from wikipedia articles
        line = re.sub(r'\u200b','', line) #removing special characters from wikipedia articles
        line = re.sub(r'[\u4e00-\u9fff]+','', line) #removing special characters from wikipedia articles
        line = "".join(c for c in line if c.isalpha() or c == " ")
        ruDataPrep.append(line)
        ruLang.append('Russian')

In [62]:
#finalizing this dataset
fullData = pd.DataFrame({
    'Text':kazDataPrep + ruDataPrep,
    'Language':kazLang + ruLang
})
fullData.to_csv(raw_datadir +'/kazrusDial.csv',index=False)

### Downloaded this dataset from open source russian dialogues dataset

In [3]:
ruJokesData = pd.read_csv('rusJokes.txt',sep = '\t', names = ['text'])
ruJokesData.to_csv('jokesData.csv')


,text
0,- Как водичка ?
1,"- А я здесь как женшина сижу, а не как термометр."
2,- Я затрудняюсь поставить вам диагноз ... Наве...
3,"- Хорошо, доктор. Я приду, когда Вы будете тре..."
4,- Что такое дефицит в маркистском понимании?
...,...
252489,- Уже в задницу!
252490,"- Да нам все равно, ехать куда?"
252491,"- Пап, я в пятницу на свадьбу иду, дай денег."
252492,- Пять тысяч хватит?


https://bilim-all.kz/quote/joke?page=11 kazakh jokes

https://www.zharar.com/kz/anekdot/